In [79]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
sw=stopwords.words('english')
ps=PorterStemmer()

In [105]:
engine = create_engine(connstr)
conn = engine.connect()
query = """ \
SELECT 
CONVERT(NVARCHAR(15) , IdText) as IDT , 
REPLACE(IdText, '.','_') as IDT_ , 
CASE 
WHEN CONVERT(NVARCHAR(9),Section_Count) = '0' THEN '1a'
WHEN CONVERT(NVARCHAR(9),Section_Count) = '1' THEN '1b'
ELSE  CONVERT(NVARCHAR(9),Section_Count) 
END AS SEC 
, CONVERT(NVARCHAR(9) , QPK) as QPK
, QTEXT as QTXT 
FROM vw_MetricsCompositeKeys WHERE PK_FORM = '2021-A-HVA'
ORDER BY Section_Count ASC, SortPos ASC
""" 
df = pd.read_sql(query,con=conn)   
df.to_excel(r'C:\Users\Tim\Documents\1802\BOD1802_2021.xlsx','AllMetrics') 
df

,IDT,IDT_,SEC,QPK,QTXT
0,,,1a,22575,HVA Information
1,1,1,1a,22577,What is the purpose of the HVA?
2,2,2,1a,22578,What services does the HVA system provide?
3,2.1,2_1,1a,22579,To whom does the HVA system provide service to?
4,3,3,1a,22803,What is the approximate time in years since de...
...,...,...,...,...,...
163,22,22,5,22755,Please list each direct interconnection or dat...
164,22.1,22_1,5,22756,Name of Connected System
165,22.2,22_2,5,22757,Is this interconnection/data exchange mission ...
166,22.3,22_3,5,22758,Is this Interconnection or data exchange a hig...


In [ ]:
df=df.loc[ df.IDT_.isin(['4_1_1','6_2_1','6_2_2','6_2_3','6_2_4','6_2_4_1','6_3_1_1' ]) ]
df

In [94]:
aspx = '' 
for i,r in df.iterrows():  
    s = open(f'C:\\_som\\_src\\TXT.aspx', 'r').read()
    s = s.replace('{0}',str(r.QPK)).replace('{1}',str(r.IDT_)).replace('{4}',str(r.IDT_)+ ' : ' +str(r.QPK)+ '\n\t' +str(r.QTXT)).replace('{5}','')
    s = s.replace('{2}',str(160))
    df.loc[i, 'ASPX'] = s
    aspx = aspx + s + '\n'
    print(df.loc[i, 'ASPX'] + '\n\n') 
  


<!-- 
    4_1_1 : 22814
	If Other:
--->
<tr id="r-m-4_1_1" class="" style="display:<%= _ShowRow_4_1_1  %>">
    <td class="LabelNoWrap" style="width: 4%; text-align: left">
        <uc1:CBlabel CSSClass="CustomControlLabelNoVertAlign" ID="CBlabel_4_1_1_i" runat="server" LabelType="QuestionNumber"
            LinkEditID="CBtext_4_1_1" />
    </td>
    <td class="LabelColumn" colspan="3" style="width: 73%; text-align: left">
        <uc1:CBlabel CSSClass="CustomControlLabelNoVertAlign" ID="CBlabel_4_1_1_ii" runat="server" LabelType="Question"
            LinkEditID="CBtext_4_1_1" /> 
        <asp:Label ID="Label_4_1_1_i" runat="server" Text="" CssClass="CustomControlLabelNoVertAlign"></asp:Label>
    </td>
    <td class="ControlColumn" style="width: 17%; text-align: right">
        <uc2:CBtext2 ID="CBtext_4_1_1" runat="server" CSSClass="CustomControlValue" EditState="ReadMode"
            IsReadOnly="false" NullDisplay="Blank" PK_Question="22814" UseSpellCheck="false" MaxLength="1500"
  

In [ ]:
with open(r'C:\\_som\\_cache.txt', 'w') as file:
  file.write(aspx)

In [60]:
conn.close()

In [93]:
t = """
Languages technologies platforms is your web don't we have web driver 
"""
def encode(t):
    t = ' '.join([w for w in t.split(' ') if w not in sw])
    t = ' '.join([ps.stem(w) for w in t.split(' ')])
    t = t.upper()
    t = re.sub('[^A-Z0-9]', '', t )   
    t = re.sub('[AEIOUY]', '', t)  
    return t
encode(t)

'LNGGTCHNLGPLTFRMWBWBDRVR'